In [1]:
from XRDXRFutils.data import SyntheticDataXRF
import h5py
import os
import glob
import numpy as np
from XRDXRFutils.Xmendeleev import Xmendeleev
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
#Labels = ['Pb-L','Fe-K','Mn-K','Cu-K','Co-K','Ca-K','Ti-K','S-K','P-K','Au-L','Al-K','Si-K','Cl-K','Sb-L','Sn-L','Sr-K','V-K']
Labels = ['Pb-L','Pb-M', 'Fe-K', 'Mn-K', 'Cu-K', 'Zn-K', 'Co-K', 'Cd-L', 'K-K',
         'Ca-K', 'Ba-L', 'Ti-K', 'As-K', 'Hg-L', 'Hg-M', 'S-K', 'P-K', 'Cr-K',
         'Au-L', 'Au-M', 'Al-K', 'Si-K', 'Cl-K', 'Se-K', 'Sn-L', 'Br-K', 'Sr-K', 'V-K','Bi-L','Bi-M',
         'Ni-K','Ag-K','Ag-L', 'Sb-L','Ar-K', 'Sc-K']
#Labels = "Pb-L Ca-K Fe-K Mn-K".split()
#Labels = ['Pb-L', 'Pb-M', 'Fe-K', 'Mn-K', 'Cu-K', 'Co-K', 'Ca-K', 'Ti-K', 'S-K',
#       'P-K', 'Au-L', 'Au-M', 'Al-K', 'Si-K', 'Cl-K', 'Sb-L', 'Sn-L', 'Sr-K', 'V-K', 'Ar-K']
symbols = []
lines = []
for l in Labels:
    ls = l.split('-')
    symbols.append(ls[0])
    lines.append(ls[1])
#atnum_list = [20,25,26,80]
#['P' 'S' 'K' 'Ca' 'Ti' 'Cr' 'Mn' 'Fe' 'Co' 'Cu' 'Zn' 'As' 'Cd' 'Ba' 'Au' 'Hg']
#symbols = ['Pb', 'Fe', 'Mn', 'Cu', 'Zn', 'Co', 'Cd', 'K', 'Ca', 'Ba', 'Ti', 'As', 'Hg', 'S', 'P', 'Cr', 'Au']
#symbols = ['Pb', 'Fe', 'Mn', 'Cu', 'Zn', 'Co', 'Cd', 'K', 'Ca', 'Ba', 'Ti', 'As', 'Hg', 'S', 'P', 'Cr', 'Au']
#symbols += ['Al', 'Si', 'Cl', 'Se', 'Sb', 'Sn', 'Br', 'Sr', 'V', 'Bi', 'Ni', 'Ag']
#symbols += ['Si', 'Cl', 'Se', 'Sn', 'Br', 'Sr', 'Bi', 'Ni']
#lines = "L K K K K K K K K L K K L K K K L".split()
#lines += "K K K K K K K K K L K K".split()
#lines += "K K K K K K L K".split()
#print([f"{s}-{l}" for s,l in zip(symbols,lines)])
print(Labels)
print(symbols)
print(lines)
print(len(symbols), len(lines))
print(["-".join(x) for x in zip(symbols,lines)])

['Pb-L', 'Pb-M', 'Fe-K', 'Mn-K', 'Cu-K', 'Zn-K', 'Co-K', 'Cd-L', 'K-K', 'Ca-K', 'Ba-L', 'Ti-K', 'As-K', 'Hg-L', 'Hg-M', 'S-K', 'P-K', 'Cr-K', 'Au-L', 'Au-M', 'Al-K', 'Si-K', 'Cl-K', 'Se-K', 'Sn-L', 'Br-K', 'Sr-K', 'V-K', 'Bi-L', 'Bi-M', 'Ni-K', 'Ag-K', 'Ag-L', 'Sb-L', 'Ar-K', 'Sc-K']
['Pb', 'Pb', 'Fe', 'Mn', 'Cu', 'Zn', 'Co', 'Cd', 'K', 'Ca', 'Ba', 'Ti', 'As', 'Hg', 'Hg', 'S', 'P', 'Cr', 'Au', 'Au', 'Al', 'Si', 'Cl', 'Se', 'Sn', 'Br', 'Sr', 'V', 'Bi', 'Bi', 'Ni', 'Ag', 'Ag', 'Sb', 'Ar', 'Sc']
['L', 'M', 'K', 'K', 'K', 'K', 'K', 'L', 'K', 'K', 'L', 'K', 'K', 'L', 'M', 'K', 'K', 'K', 'L', 'M', 'K', 'K', 'K', 'K', 'L', 'K', 'K', 'K', 'L', 'M', 'K', 'K', 'L', 'L', 'K', 'K']
36 36
['Pb-L', 'Pb-M', 'Fe-K', 'Mn-K', 'Cu-K', 'Zn-K', 'Co-K', 'Cd-L', 'K-K', 'Ca-K', 'Ba-L', 'Ti-K', 'As-K', 'Hg-L', 'Hg-M', 'S-K', 'P-K', 'Cr-K', 'Au-L', 'Au-M', 'Al-K', 'Si-K', 'Cl-K', 'Se-K', 'Sn-L', 'Br-K', 'Sr-K', 'V-K', 'Bi-L', 'Bi-M', 'Ni-K', 'Ag-K', 'Ag-L', 'Sb-L', 'Ar-K', 'Sc-K']


In [3]:
xm = Xmendeleev()
#atnum_list = sorted([xm.get_element(s).atomic_number for s in symbols[1:]])
atnum_list = sorted(set([xm.get_element(s).atomic_number for s in symbols]))
print(atnum_list)

[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 38, 47, 48, 50, 51, 56, 79, 80, 82, 83]


In [4]:
def rem_nan(x):
    idx = lambda y: np.where(np.isnan(y[0]).sum(axis=1) == 0)[0]
    x.reflayer_thickness = x.reflayer_thickness[idx(x.labels)]
    x.sublayer_thickness = x.sublayer_thickness[idx(x.labels)]
    x.time = x.time[idx(x.labels)]
    x.weight_fractions = x.weight_fractions[idx(x.labels), :]
    x.data = x.data[:, idx(x.labels), :]
    x.labels = x.labels[:, idx(x.labels), :]
    return x

In [5]:
def saveh5(rootdir, fname = "data", fout = None, single = False):
    if rootdir[-1] == "/":
        rootdir = rootdir[:-1]
    datadir = os.path.join(rootdir, 'outdata')
    synt = SyntheticDataXRF(rl_atnum_list=atnum_list, skip_element=True)
    synt.read(datadir)
    print("Done")
    synt.get_data_and_labels(symbols, lines)
    if single:
        synt.sublayer_thickness = np.zeros_like(synt.reflayer_thickness)
    if not fout:
        fout = os.path.join(rootdir, f"{os.path.basename(rootdir)}_{fname}.h5")
    if np.any(np.isnan(synt.labels)):
        synt = rem_nan(synt)
    synt.save_h5(fout)
    print("Saved")
    return fout

def save_resampled(h5fname, nbins = 1024, bounds = (0,30)):
    print('resampling...')
    synt = SyntheticDataXRF().load_h5(h5fname)
    s = synt.resample(nbins = nbins, bounds = bounds)
    synt.data = s.data
    synt.labels = s.labels
    synt._x = s.x
    outfname = h5fname[:-3] + f'_{nbins}.h5'
    synt.save_h5(outfname)
    print('Done')
    return outfname

def saveh5_from_file(h5file_name : str, xmso_datadirs : list):
    synt = SyntheticDataXRF(rl_atnum_list=atnum_list, skip_element=True)
    synt.spe_objs = []
    for _dir in xmso_datadirs:
        xmso_fnames = glob(os.path.join(_dir), "*.xmso")
        print(f"Reading data from {_dir}")
        synt.spe_objs += [s for s in synt.__read__(xmso_fnames)]
        print("Done")
    synt.get_data_and_labels(symbols,lines)
    synt.save_h5(h5file_name)
    print("Saved")

def joinh5(h5list, out_file_name = None):
    data = []
    unconv_data = []
    labels = []
    reflayer_thickness = []
    sublayer_thickness = []
    #reflayer_elements = []
    weight_fractions = []
    total_counts = []
    energy = None
    #time = []
    metadata = {}
    metadata["path"] = []
    for i, h5 in enumerate(h5list):
        synt = SyntheticDataXRF(rl_atnum_list=atnum_list, skip_element=True)
        synt.load_h5(h5)
        if isinstance(synt.metadata['path'], str):
            synt.metadata['path'] = [synt.metadata['path']]
        data.append(synt.data)
        labels.append(synt.labels)
        reflayer_thickness.append(synt.reflayer_thickness)
        sublayer_thickness.append(synt.sublayer_thickness)
        #reflayer_elements.append(synt.reflayer_elements)
        weight_fractions.append(synt.weight_fractions)
        total_counts.append(synt.total_counts)
        energy = synt.x
        #time.append(synt.time)
        metadata["path"] += [m for m in synt.metadata['path']]
    out = SyntheticDataXRF(rl_atnum_list=atnum_list, skip_element=True)
    out.data = np.concatenate(data, axis = 1)
    out.unconv_data = np.concatenate(unconv_data, axis = 1)
    out.labels = np.concatenate(labels, axis = 1)
    out.reflayer_thickness = np.hstack(reflayer_thickness)
    out.sublayer_thickness = np.hstack(sublayer_thickness)
    #out.reflayer_elements = np.vstack(reflayer_elements)
    out.weight_fractions = np.vstack(weight_fractions)
    out.total_counts = np.hstack(total_counts)
    out._x = energy
    #out.time = np.hstack(time)
    out.metadata.update(synt.metadata)
    out.metadata["path"] = metadata["path"]
    print(out.metadata)
    out.spe_objs = []
    print(out.shape)
    if not out_file_name:
        out_file_name = "joined_data.h5"
    out.save_h5(out_file_name)
    

In [ ]:
rootdir = '/home/rosario/xmimsim/synthetic/slfixed_rl_30_90/'
f = saveh5(rootdir, fout = '/home/rosario/test_unconv_spec.h5')
f = save_resampled(f)

Reading SyntXRF data from /home/rosario/xmimsim/synthetic/slfixed_rl_30_90/outdata


In [38]:
joinh5([
    '/home/rosario/xmimsim/synthetic/slfixed_Rh_231max_nakagami_Pb_rl_1_90/slfixed_Rh_231max_nakagami_Pb_rl_1_90_data_1024.h5',
    '/home/rosario/xmimsim/synthetic/zdenek/slfixed_PbO2_Rh231max_nakagami_1_90_data_1024.h5'
], '/home/shared/syntdataXRF/slfixed_PbO2_Rh231max_nakagami_1_90_data_joined_1024.h5')

Loading: /home/rosario/xmimsim/synthetic/slfixed_Rh_231max_nakagami_Pb_rl_1_90/slfixed_Rh_231max_nakagami_Pb_rl_1_90_data_1024.h5
Loading: /home/rosario/xmimsim/synthetic/zdenek/slfixed_PbO2_Rh231max_nakagami_1_90_data_1024.h5
{'labels': array(['Pb-L', 'Ca-K', 'Fe-K', 'Mn-K', 'time'], dtype=object), 'notes': 'weight fractions columns ordered like reflayer_elements', 'path': ['/home/rosario/xmimsim/synthetic/slfixed_Rh_231max_nakagami_Pb_rl_1_90/outdata', '/home/shared/slfixed_PbO2_Rh231max_nakagami_1_90/outdata'], 'reflayer_elements': array(['Ca', 'Mn', 'Fe', 'Pb'], dtype=object)}
(1, 4240, 1024)
Saving: /home/shared/syntdataXRF/slfixed_PbO2_Rh231max_nakagami_1_90_data_joined_1024.h5


In [23]:
#synt = SyntheticDataXRF().load_h5(f)
synt = SyntheticDataXRF().load_h5('/home/shared/syntdataXRF/Mo_amuleti_rl_1_50_Si_fixed_joined_data_1024.h5')
print(synt.metadata)
print(synt.shape)
print(synt.labels.shape)
print('wf', synt.weight_fractions.shape)
print(np.unique(synt.sublayer_thickness))
print('total_counts: ',synt.total_counts.min(), synt.total_counts.max())
print('wf min: ',synt.weight_fractions.min(axis = 0))
print('wf max: ', synt.weight_fractions.max(axis = 0))
print('wfpb:', synt.weight_fractions[:,-2].max())
print('min Pb: ', synt.labels[0,:,10].min())
print('max Pb: ', synt.labels[0,:,10].max())
print('rl at max Pb: ', synt.reflayer_thickness[np.argmax(synt.labels[0,:,0])]*1e4)
print('rl min', synt.reflayer_thickness.min())
print('rl max', synt.reflayer_thickness.max())
print('sl thickness shape: ', synt.sublayer_thickness.shape)
print('sl thickness:', np.unique(synt.sublayer_thickness))
print(hasattr(synt, 'time'))

Loading: /home/shared/syntdataXRF/Mo_amuleti_rl_1_50_Si_fixed_joined_data_1024.h5
{'labels': array(['Pb-L', 'Pb-M', 'Fe-K', 'Mn-K', 'Cu-K', 'Zn-K', 'Co-K', 'Cd-L',
       'K-K', 'Ca-K', 'Ba-L', 'Ti-K', 'As-K', 'Hg-L', 'Hg-M', 'S-K',
       'P-K', 'Cr-K', 'Au-L', 'Au-M', 'Al-K', 'Si-K', 'Cl-K', 'Se-K',
       'Sn-L', 'Br-K', 'Sr-K', 'V-K', 'Bi-L', 'Bi-M', 'Ni-K', 'Ag-K',
       'Ag-L', 'Sb-L', 'Ar-K', 'Sc-K', 'time'], dtype=object), 'notes': 'weight fractions columns ordered like reflayer_elements', 'path': array(['/home/rosario/xmimsim/synthetic/SiO2_new_template_Siwf_not_shuffled_rl_1_50/outdata',
       '/home/shared/Mo_amuleti_1_50/outdata'], dtype=object), 'reflayer_elements': array(['Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr',
       'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'As', 'Se', 'Br', 'Sr', 'Ag',
       'Cd', 'Sn', 'Sb', 'Ba', 'Au', 'Hg', 'Pb', 'Bi'], dtype=object)}
(1, 11166, 1024)
(1, 11166, 37)
wf (11166, 31)
[0.01]
total_counts:  9.538245675899999 1760